# solkogan

https://github.com/solkogan/searchface

Аналог Findface для небольших городов

Система состоит из нескольких скриптов - скачивает фото юзеров из профилей ВК в рамках одного указанного города, и создает базу биометрических данных этих лиц, связывая их с аккаунтами ВК.

Скрипты протестированы на Parrot OS и требуют установки модулей Python которые есть в разделах import у скриптов

Для того чтобы скрипты заработали клонируйте этот репозиторий и удалите из подкаталогов файлы README.MD - иначе они будут мешать работе скриптов

Скачайте этот файл https://cloud.mail.ru/public/2KGj/2pWSDbXZt и бросьте в папку с проектом

VkIdsParser_1.py - впишите в скрипт ваш логин пароль ВК (с выключенной двухфакторной авторизацией)
Также поменяйте параметры аккаунтов которые надо парсить - пол, возраст, город

По итогам работы скрипта вы получите файлик ids.txt со списком ID ВК

DownloadPhotosToJpg_2.py - впишите в скрипт ваш логин пароль ВК (с выключенной двухфакторной авторизацией)
скрипт берет данные из списка ids.txt и скачивает в папку jpg по нескольку фото с каждого аккаунта

Имена файлов ссответствуют id ВК

JpgToNpy_3.py - Производит процесс сбора биометрических параметров лиц с фото в папке jpg и сохраняет их в паку npy в виде мелких файликов
При больших объемах фоток скрипт может работать достаточно долго (часы, дни)

Тем не менее его можно выключить в любой момент а при следующем запуске он начнет с того места где остановился

FindIntoNpy_4.py - скрипт пытается найти среди файлов .npy в папке npy биометрию схожую с 1.jpg
Файл 1.jpg с искомым лицом нужно предварительно поместить в корневую директорию проекта

Данный скрипт ищет очень медленно из-за того что ему нужно перебрать все файлы .npy

Не рекомендуется использовать данный способ для поиска - только для проверки корректности создания .npy

NpyToEmbeddingsBin_5.py - скрипт берет все файлы .npy из папки npy и создает на их основе файл embeddings.bin
Созданный файл по сути содержит биометрию всех ранее распознанных лиц и все файлы с папок jpg и npy после его создания уже не нужны

Помимо этого создается файл associations.txt в котором хранятся соответствия номеру записи в эмбединге и имени файла .npy из которого она была взята

FinfFaceInEmbeddingsBin_6.py - пытается найти в базе лицо схожее с лицом на 1.jpg
Файл 1.jpg с искомым лицом нужно предварительно поместить в корневую директорию проекта

В отличии от скрипта номер 4 работает очень быстро.

In [4]:
#pip install vk_api

# VkIdsParser_1.py

In [5]:
import vk_api
import time, codecs
import os, sys, shutil, requests
import os.path
import datetime

In [16]:

def findAndWriteUsers(
    citynumber
    , byear
    , vk
    , rewrite):
    # Сгенерировать все даты за год byear
    # 
    
    start = datetime.datetime.strptime("01-01-" + str(byear), "%d-%m-%Y")
    end = datetime.datetime.strptime("31-12-"+ str(byear), "%d-%m-%Y")
    date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

    
    
    # Режим открытия файла.
    if rewrite:
        f_mode = "w"
    else:
        f_mode = "r"
    

    # Открываем файл для записи результатов (дозапись)
    ff = codecs.open('ids' + str(citynumber) + '.txt', f_mode, encoding='utf8')

    # Открываем файл для записи людей без фоток  (дозапись)
    fwof = codecs.open('ids' + str(citynumber) + '_wof.txt', f_mode, encoding='utf8')

    
    # Перебор возрастов
    for date in date_generated:
        #print (date.strftime("%d-%m-%Y"))
        #print (date.strftime("%Y-%m-%d"))
    #    print (date.strftime("%Y"))

        try:

            # Такой вот механизм завершения работы
            if not(os.path.exists('searchface_flag.txt')):
                with open('searchface_datset.txt', 'w') as setf:
                    setf.write(str(date))
                break

            psex=1
            while(psex<=2):

                # Пауза для API
                time.sleep(4)

                byear = date.strftime("%Y")
                bmonth = date.strftime("%m")
                bday = date.strftime("%d")



                # Пишем какую группу людей качаем
                #print('Download ID: '+ str(vozrast) + ' age, born in ' + str(mesac))
                #rint('Date: '+ date.strftime("%d-%m-%Y") + ', sex: ' + str(psex))

                # Получаем 1000 юзеров - их ФИО, айди, и фотку 
                #z = vk.users.search(count=1000, fields='id, photo_max_orig, has_photo, first_name, last_name', city=citynumber, sex=psex, age_from=vozrast, age_to=vozrast, birth_month=mesac)

                z = vk.users.search(count=1000
                        , fields='id, photo_max_orig, has_photo, first_name, last_name'
                        , city=citynumber, sex=psex, birth_year=byear, birth_month=bmonth, birth_day=bday)

                print('Date: '+ date.strftime("%d-%m-%Y") + ', sex: ' + str(psex) + '. Peoples count: ' + str(z['count']))

                for x in z['items']:
                    if(x['has_photo']==1):
                        # Записываем данные о юзере в файл разделяя черточкой |
                        s = date.strftime("%Y-%m-%d")+'|'+str(x['id'])+'|'+str(x['photo_max_orig'])+'|'+str(x['first_name'])+' '+str(x['last_name'])+'\n'
                        ff.write(s)
                    else:
                        s = date.strftime("%Y-%m-%d")+'|'+str(x['id'])+'|'+str(x['photo_max_orig'])+'|'+str(x['first_name'])+' '+str(x['last_name'])+'\n'
                        fwof.write(s)
                psex += 1
        except Exception as e:
            print(str(e))
            ff.close()
            fwof.close()
            with open('searchface_datset.txt', 'w') as setf:
                setf.write(str(date))
            break
        #finally:
            #print('')

    ff.close()
    fwof.close()
    print('Done!')

In [13]:
# Заходим ВКонтакте под своим логином
vk_session = vk_api.VkApi('79002608050', 'BdFX1UZdvAu3vu')
vk_session.auth()
vk = vk_session.get_api()

In [ ]:
findAndWriteUsers(
    72     #citynumber
    , 1900 #byear
    , vk
    , 1    #rewrite
    )

Date: 01-01-1900, sex: 1. Peoples count: 1
Date: 01-01-1900, sex: 2. Peoples count: 1
Date: 02-01-1900, sex: 1. Peoples count: 0
Date: 02-01-1900, sex: 2. Peoples count: 0
Date: 03-01-1900, sex: 1. Peoples count: 0
Date: 03-01-1900, sex: 2. Peoples count: 0
Date: 04-01-1900, sex: 1. Peoples count: 0
Date: 04-01-1900, sex: 2. Peoples count: 0
Date: 05-01-1900, sex: 1. Peoples count: 0
Date: 05-01-1900, sex: 2. Peoples count: 0
Date: 06-01-1900, sex: 1. Peoples count: 0
Date: 06-01-1900, sex: 2. Peoples count: 0
Date: 07-01-1900, sex: 1. Peoples count: 0
Date: 07-01-1900, sex: 2. Peoples count: 0
Date: 08-01-1900, sex: 1. Peoples count: 0
Date: 08-01-1900, sex: 2. Peoples count: 0
Date: 09-01-1900, sex: 1. Peoples count: 0
Date: 09-01-1900, sex: 2. Peoples count: 0
Date: 10-01-1900, sex: 1. Peoples count: 0
Date: 10-01-1900, sex: 2. Peoples count: 0
Date: 11-01-1900, sex: 1. Peoples count: 0
Date: 11-01-1900, sex: 2. Peoples count: 0
Date: 12-01-1900, sex: 1. Peoples count: 0
Date: 12-01

Date: 06-04-1900, sex: 2. Peoples count: 0
Date: 07-04-1900, sex: 1. Peoples count: 0
Date: 07-04-1900, sex: 2. Peoples count: 0
Date: 08-04-1900, sex: 1. Peoples count: 0
Date: 08-04-1900, sex: 2. Peoples count: 0
Date: 09-04-1900, sex: 1. Peoples count: 0
Date: 09-04-1900, sex: 2. Peoples count: 0
Date: 10-04-1900, sex: 1. Peoples count: 0
Date: 10-04-1900, sex: 2. Peoples count: 0
Date: 11-04-1900, sex: 1. Peoples count: 0
Date: 11-04-1900, sex: 2. Peoples count: 0
Date: 12-04-1900, sex: 1. Peoples count: 0
Date: 12-04-1900, sex: 2. Peoples count: 0
Date: 13-04-1900, sex: 1. Peoples count: 0
